In [ ]:
from RandomTesting.__RandomTesting__ import Random_Testing, calculate_extract_loss
from sklearn import neighbors
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

# Load Data

In [ ]:
# 2021/02/02 OK

sta_lat=[9,61,66,70,70,74,74,76,77,81,81,82,87,89,89,
         102,112,118,118,125,144,160,160,169,185,185,
         189,189,189,205,206,210,211,220,223,227,229,
         230,236,239,252,270,277,283,288,288,294,300,
         304,309,309,311,312,313,315,315,316,317,317,
         318,319,320,320,320,320,320,321,321,324,326,
         330,331,332]

sta_lon=[79,42,57,34,43,31,32,36,29,30,49,34,33,31,116,
         54,21,22,117,30,32,25,45,35,21,55,26,35,68,69,
         41,97,160,55,68,47,65,62,57,75,76,83,180,90,109,
         175,98,122,104,121,123,154,146,131,146,152,153,
         109,144,151,158,121,149,152,153,165,137,150,152,
         177,145,169,153]

EPA73_test_pm25 = np.load('EPA73_Testing_348X204_PM25.npy', mmap_mode='r')
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

# Make KNN File

In [ ]:
K_nb = 7
for extract_nb in range(1,5):
    EPA73_test_fill=[]
    for qq in range(30):
        matrix20, lats_ex20, lons_ex20 = Random_Testing(extract_nb, qq, station_coordinate, EPA73_test_pm25, 
                                                        ex5_lst, ex10_lst, ex15_lst, ex20_lst)

        total_result = []
        test_x = ([[i,j] for i in range(348) for j in range(204)])
        input_data = matrix20
        for i in tqdm(range(input_data.shape[0])):
            value_lst, result = ([] for _ in range(2))

            lat_all = (np.where(input_data[i,:,:,0] != 0)[0])
            lon_all = (np.where(input_data[i,:,:,0] != 0)[1])

            for lat, lon in zip(lat_all, lon_all): value_lst.append(input_data[i,lat,lon,0])

            train_x = np.vstack((lat_all,lon_all)).T
            train_y = np.array(value_lst,dtype='float32')

            knn = neighbors.KNeighborsRegressor(K_nb, weights = 'distance')

            knn_fit = knn.fit(train_x, train_y)
            test_y = knn_fit.predict(test_x)
            for k in range(348): result.append(test_y[k*204:(k+1)*204])

            total_result.append(result)
            
            del result, test_y
            gc.collect()
        EPA73_test_fill.append(total_result)
    np.save('EPA73_testing_KNN'+str(K_nb)+'fill_ex'+str(extract_nb*5)+'_without_mou',EPA73_test_fill)
    print('*'*50)
    del EPA73_test_fill, matrix20

# KNN3 BaseLine

In [24]:
ex5_KNN = np.load('EPA73_testing_KNN7fill_ex5_without_mou.npy', mmap_mode='r')
ex10_KNN = np.load('EPA73_testing_KNN7fill_ex10_without_mou.npy', mmap_mode='r')
ex15_KNN = np.load('EPA73_testing_KNN7fill_ex15_without_mou.npy', mmap_mode='r')
ex20_KNN = np.load('EPA73_testing_KNN7fill_ex20_without_mou.npy', mmap_mode='r')

In [28]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex5,lons_ex5=Random_Testing(1,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex5_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex5, lons_ex5)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('max:  '+str(np.round(np.array(total_mse),4).max()))
print('min:  '+str(np.round(np.array(total_mse),4).min()))
print('mean: '+str(np.round(np.array(total_mse),4).mean()))
print('var:  '+str(np.round(np.array(total_mse),4).var()))
print('std:  '+str(np.round(np.array(total_mse),4).std()))

100%|██████████| 30/30 [23:38<00:00, 62.49s/it]

max:  113.2738
min:  23.1724
mean: 42.847016666666654
var:  503.4674577087223
std:  22.43808052638911


In [27]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex10,lons_ex10=Random_Testing(2,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex10_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex10, lons_ex10)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('max:  '+str(np.round(np.array(total_mse),4).max()))
print('min:  '+str(np.round(np.array(total_mse),4).min()))
print('mean: '+str(np.round(np.array(total_mse),4).mean()))
print('var:  '+str(np.round(np.array(total_mse),4).var()))
print('std:  '+str(np.round(np.array(total_mse),4).std()))

100%|██████████| 30/30 [25:48<00:00, 54.49s/it]

max:  87.343
min:  25.9978
mean: 37.273726666666676
var:  228.48376763662225
std:  15.115679529436388


In [26]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex15,lons_ex15=Random_Testing(3,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex15_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex15, lons_ex15)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('max:  '+str(np.round(np.array(total_mse),4).max()))
print('min:  '+str(np.round(np.array(total_mse),4).min()))
print('mean: '+str(np.round(np.array(total_mse),4).mean()))
print('var:  '+str(np.round(np.array(total_mse),4).var()))
print('std:  '+str(np.round(np.array(total_mse),4).std()))

100%|██████████| 30/30 [29:39<00:00, 51.82s/it]

max:  68.7478
min:  30.1125
mean: 40.52830666666667
var:  88.97460557062223
std:  9.432635133970901


In [25]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex20,lons_ex20=Random_Testing(4,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex20_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex20, lons_ex20)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('max:  '+str(np.round(np.array(total_mse),4).max()))
print('min:  '+str(np.round(np.array(total_mse),4).min()))
print('mean: '+str(np.round(np.array(total_mse),4).mean()))
print('var:  '+str(np.round(np.array(total_mse),4).var()))
print('std:  '+str(np.round(np.array(total_mse),4).std()))

100%|██████████| 30/30 [26:33<00:00, 51.06s/it]

max:  63.0025
min:  30.8471
mean: 45.06712666666667
var:  90.46209921462221
std:  9.511156565561425


# KNN5 BaseLine

In [ ]:
EPA73_test_pm25 = np.load('../Data/EPA73_Testing_348X204_PM25.npy', mmap_mode='r')[:3000]
ex5_KNN = np.load('EPA73_testing_KNN5fill_ex5_without_mou.npy', mmap_mode='r')
ex10_KNN = np.load('EPA73_testing_KNN5fill_ex10_without_mou.npy', mmap_mode='r')
ex15_KNN = np.load('EPA73_testing_KNN5fill_ex15_without_mou.npy', mmap_mode='r')
ex20_KNN = np.load('EPA73_testing_KNN5fill_ex20_without_mou.npy', mmap_mode='r')

station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex5,lons_ex5=Random_Testing(1,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex5_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex5, lons_ex5)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract5 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex10,lons_ex10=Random_Testing(2,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex10_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex10, lons_ex10)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract10 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex15,lons_ex15=Random_Testing(3,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex15_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex15, lons_ex15)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract15 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex20,lons_ex20=Random_Testing(4,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex20_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex20, lons_ex20)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract20 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))

# KNN7 BaseLine

In [ ]:
EPA73_test_pm25 = np.load('../Data/EPA73_Testing_348X204_PM25.npy', mmap_mode='r')[:3000]
ex5_KNN = np.load('EPA73_testing_KNN7fill_ex5_without_mou.npy', mmap_mode='r')
ex10_KNN = np.load('EPA73_testing_KNN7fill_ex10_without_mou.npy', mmap_mode='r')
ex15_KNN = np.load('EPA73_testing_KNN7fill_ex15_without_mou.npy', mmap_mode='r')
ex20_KNN = np.load('EPA73_testing_KNN7fill_ex20_without_mou.npy', mmap_mode='r')

station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex5,lons_ex5=Random_Testing(1,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex5_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex5, lons_ex5)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract5 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex10,lons_ex10=Random_Testing(2,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex10_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex10, lons_ex10)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract10 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex15,lons_ex15=Random_Testing(3,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex15_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex15, lons_ex15)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract15 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))

In [ ]:
total_mse, total_mae, total_mape = ([] for _ in range(3))
for i in tqdm(range(30)):
    _ ,lats_ex20,lons_ex20=Random_Testing(4,i,station_coordinate, EPA73_test_pm25, ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    mse, mae, mape = calculate_extract_loss(ex20_KNN[i], EPA73_test_pm25[:,:,:,0], lats_ex20, lons_ex20)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)
print('extract20 mean :',np.mean(total_mse),np.mean(total_mae),np.mean(total_mape))